In [1]:
# conda install -c conda-forge lightgbm

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
import pickle

C:\Users\hmitt\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
train = pd.read_csv("C:\\Users\\hmitt\\OneDrive\\Documents\\MSIS\\Sem_2_Spring_2023\\Projects\\NewYork-taxi-fare-prediction\\train_FE.csv")
train

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_hour,...,pickup_year,pickup_day_of_week,hav_distance,euc_distance,precipitation,snow_depth,snowfall,max_temp,min_temp,avg_wind
0,0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1,17,...,2009,Monday,0.640487,0.651073,127,0,0,222,139,14
1,1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1,16,...,2010,Tuesday,5.250670,5.499001,0,0,0,-11,-67,44
2,2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,0,...,2011,Thursday,0.863411,0.943509,18,0,0,283,194,22
3,3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1,4,...,2012,Saturday,1.739386,1.748493,56,0,0,217,122,22
4,4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,7,...,2010,Tuesday,1.242218,1.343397,0,0,0,161,67,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955579,1955579,2014-11-17 13:51:02.0000002,4.0,2014-11-17 13:51:02,-73.950787,40.770836,-73.951043,40.774543,1,13,...,2014,Monday,0.256479,0.256392,391,0,0,111,44,17
1955580,1955580,2013-02-25 17:51:28.0000004,7.0,2013-02-25 17:51:28,-73.983035,40.748276,-73.974725,40.741978,1,17,...,2013,Monday,0.615289,0.719458,0,0,0,72,0,21
1955581,1955581,2013-07-12 22:03:41.0000006,10.5,2013-07-12 22:03:41,-74.003482,40.743577,-73.976758,40.765265,1,22,...,2013,Friday,2.049851,2.374785,64,0,0,250,200,43
1955582,1955582,2009-01-17 09:35:37.0000002,10.9,2009-01-17 09:35:37,-73.971819,40.797243,-73.985921,40.752459,3,9,...,2009,Saturday,3.181034,3.239675,0,0,0,-56,-144,25


In [4]:
train=train.drop('Unnamed: 0', axis=1)

In [5]:
train

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_hour,pickup_day,...,pickup_year,pickup_day_of_week,hav_distance,euc_distance,precipitation,snow_depth,snowfall,max_temp,min_temp,avg_wind
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1,17,15,...,2009,Monday,0.640487,0.651073,127,0,0,222,139,14
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1,16,5,...,2010,Tuesday,5.250670,5.499001,0,0,0,-11,-67,44
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,0,18,...,2011,Thursday,0.863411,0.943509,18,0,0,283,194,22
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1,4,21,...,2012,Saturday,1.739386,1.748493,56,0,0,217,122,22
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,7,9,...,2010,Tuesday,1.242218,1.343397,0,0,0,161,67,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955579,2014-11-17 13:51:02.0000002,4.0,2014-11-17 13:51:02,-73.950787,40.770836,-73.951043,40.774543,1,13,17,...,2014,Monday,0.256479,0.256392,391,0,0,111,44,17
1955580,2013-02-25 17:51:28.0000004,7.0,2013-02-25 17:51:28,-73.983035,40.748276,-73.974725,40.741978,1,17,25,...,2013,Monday,0.615289,0.719458,0,0,0,72,0,21
1955581,2013-07-12 22:03:41.0000006,10.5,2013-07-12 22:03:41,-74.003482,40.743577,-73.976758,40.765265,1,22,12,...,2013,Friday,2.049851,2.374785,64,0,0,250,200,43
1955582,2009-01-17 09:35:37.0000002,10.9,2009-01-17 09:35:37,-73.971819,40.797243,-73.985921,40.752459,3,9,17,...,2009,Saturday,3.181034,3.239675,0,0,0,-56,-144,25


In [6]:
# Considering all the Features except 'Key', 'pickup_date', 'pickup_day_of_week' and 'euc_distance' (as haversian distance is already being considered, and both together could lead to redundant features.)
X = train[train.columns[3:]]
X = X[X.columns.difference(['pickup_day_of_week'])]
X = X[X.columns.difference(['pickup_date'])]
X = X[X.columns.difference(['euc_distance'])]

# Trainoutput is the Fare Amount column
y = train[train.columns[1:2]]

In [7]:
X

,avg_wind,dropoff_latitude,dropoff_longitude,hav_distance,max_temp,min_temp,passenger_count,pickup_day,pickup_hour,pickup_latitude,pickup_longitude,pickup_month,pickup_year,precipitation,snow_depth,snowfall
0,14,40.712278,-73.841610,0.640487,222,139,1,15,17,40.721319,-73.844311,6,2009,127,0,0
1,44,40.782004,-73.979268,5.250670,-11,-67,1,5,16,40.711303,-74.016048,1,2010,0,0,0
2,22,40.750562,-73.991242,0.863411,283,194,2,18,0,40.761270,-73.982738,8,2011,18,0,0
3,22,40.758092,-73.991567,1.739386,217,122,1,21,4,40.733143,-73.987130,4,2012,56,0,0
4,20,40.783762,-73.956655,1.242218,161,67,1,9,7,40.768008,-73.968095,3,2010,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955579,17,40.774543,-73.951043,0.256479,111,44,1,17,13,40.770836,-73.950787,11,2014,391,0,0
1955580,21,40.741978,-73.974725,0.615289,72,0,1,25,17,40.748276,-73.983035,2,2013,0,0,0
1955581,43,40.765265,-73.976758,2.049851,250,200,1,12,22,40.743577,-74.003482,7,2013,64,0,0
1955582,25,40.752459,-73.985921,3.181034,-56,-144,3,17,9,40.797243,-73.971819,1,2009,0,0,0


In [8]:
y

,fare_amount
0,4.5
1,16.9
2,5.7
3,7.7
4,5.3
...,...
1955579,4.0
1955580,7.0
1955581,10.5
1955582,10.9


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1368908, 16) (1368908, 1)
(586676, 16) (586676, 1)


* We can use 3 Error Metrics to evaluate our models.<br>
    ○ Root Mean Square Error (RMSE)<br>
    ○ Mean Squared Error (MSE)<br>
    ○ R2 Score

* Our goal is to try to maximize R2 score (1 being the best) while minimizing RMSE and MSE along with ensuring that the model does not overfit.

### Linear regression

In [10]:
lr= LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [11]:
# perform k-fold cross-validation
scores = cross_val_score(lr, X_train, y_train, cv=6)

# predict on test set and calculate metrics
lr_y_pred = cross_val_predict(lr, X_test, y_test, cv=6)

In [12]:
# The coefficients
print('Coefficients: \n', lr.coef_)

# ERROR METRICS

# Root Mean Square Error
rmse = np.sqrt(mean_squared_error(y_test, lr_y_pred))
print("RMSE: %f" % (rmse))

# The Mean Squared Error
print("Mean squared error: %.2f" % mean_squared_error(y_test, lr_y_pred))

# R2 Score
print('Variance score: %.2f' % r2_score(y_test, lr_y_pred))

Coefficients: 
 [[ 7.07495023e-05 -1.75492997e+01 -3.13891126e+00  3.57948772e+00
  -9.40418065e-04  1.69648290e-04  3.06535298e-02  6.95892979e-04
   8.78692895e-03  4.64082226e+00  1.75268391e+01  7.57936286e-02
   5.05588976e-01  3.45709668e-04  8.83268432e-05 -9.26003748e-04]]
RMSE: 4.515310
Mean squared error: 20.39
Variance score: 0.77


* Linear regression model achieved an RMSE = 4.51, MSE = 20.39 and R2 score = 0.77.

* For a basic linear model (that could be considered as the baseline model) I believe that it did quite okay providing an RMSE = 4.62 on the Test Data. But, this is definitely not a good model and has to be improved upon.

* Also, I think for data as sparse and vast as that we have, we may not be able to model it linearly as it involves a lot of real world parameters and in general might contain lots of outliers along with many other underlying dependencies among the features. Hence, linear regression did not produce that good a result for the problem.

### Decision Tree regressor

In [13]:
dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)

DecisionTreeRegressor()

In [14]:
# perform k-fold cross-validation
scores = cross_val_score(dt, X_train, y_train, cv=6)

# predict on test set and calculate metrics
dt_y_pred = cross_val_predict(dt, X_test, y_test, cv=6)

In [15]:
# ERROR METRICS

# Root Mean Square Error
rmse_dt = np.sqrt(mean_squared_error(y_test, dt_y_pred))
print("RMSE: %f" % (rmse_dt))

# The Mean Squared Error
print("Mean squared error: %.2f" % mean_squared_error(y_test, dt_y_pred))

# R2 Score
print('Variance score: %.2f' % r2_score(y_test, dt_y_pred))

RMSE: 5.224722
Mean squared error: 27.30
Variance score: 0.69


### Random Forest regressor

In [16]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

C:\Users\hmitt\AppData\Local\Temp\ipykernel_28892\3689953576.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


RandomForestRegressor()

In [17]:
rf_y_pred = rf.predict(X_test)

In [18]:
# ERROR METRICS

# Root Mean Square Error
rmse_rf = np.sqrt(mean_squared_error(y_test, rf_y_pred))
print("RMSE: %f" % (rmse_rf))

# The Mean Squared Error
print("Mean squared error: %.2f" % mean_squared_error(y_test, rf_y_pred))

# R2 Score
print('Variance score: %.2f' % r2_score(y_test, rf_y_pred))

RMSE: 3.573594
Mean squared error: 12.77
Variance score: 0.86


### XGBoost regressor

In [19]:
xgbr = xgb.XGBRegressor()
xgbr.fit(X_train, y_train)

C:\Users\hmitt\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=20,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [20]:
# perform k-fold cross-validation
scores = cross_val_score(xgbr, X_train, y_train, cv=6)

# predict on test set and calculate metrics
xgbr_y_pred = cross_val_predict(xgbr, X_test, y_test, cv=6)

C:\Users\hmitt\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\hmitt\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\hmitt\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\hmitt\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

In [21]:
# ERROR METRICS

# Root Mean Square Error
rmse_xgbr = np.sqrt(mean_squared_error(y_test, xgbr_y_pred))
print("RMSE: %f" % (rmse_xgbr))

# The Mean Squared Error
print("Mean squared error: %.2f" % mean_squared_error(y_test, xgbr_y_pred))

# R2 Score
print('Variance score: %.2f' % r2_score(y_test, xgbr_y_pred))

RMSE: 3.562385
Mean squared error: 12.69
Variance score: 0.86


In [22]:
#Hyperparameter tuning

#set parameters for xgboost
params = {'max_depth':[7,8,9,10],
          'eta':[1],
          'silent':[1],
          'objective':['reg:linear'],
          'eval_metric':['rmse'],
          'learning_rate':[0.1,0.15,0.2]
         }

# Create the grid search object
grid_search = GridSearchCV(estimator=xgbr, param_grid=params, cv=5, n_jobs=-1)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

xgbr2_y_pred = grid_search.predict(X_test)

C:\Users\hmitt\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[22:47:23] WARNING: c:\windows\temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[22:47:23] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




In [23]:
# ERROR METRICS

# Root Mean Square Error
rmse_xgbr2 = np.sqrt(mean_squared_error(y_test, xgbr2_y_pred))
print("RMSE: %f" % (rmse_xgbr2))

# The Mean Squared Error
print("Mean squared error: %.2f" % mean_squared_error(y_test, xgbr2_y_pred))

# R2 Score
print('Variance score: %.2f' % r2_score(y_test, xgbr2_y_pred))

RMSE: 3.464686
Mean squared error: 12.00
Variance score: 0.86


### Light GBM regressor

In [24]:
lgbm = LGBMRegressor (boosting_type='gbdt', num_leaves=31, learning_rate=0.1, max_depth=5, n_estimators=100, silent = True)
lgbm.fit (X_train, y_train)

C:\Users\hmitt\anaconda3\lib\site-packages\lightgbm\sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMRegressor(max_depth=5, silent=True)

In [25]:
lgbm_y_predict = lgbm.predict(X_test)

In [26]:
# ERROR METRICS

# Root Mean Square Error
rmse_lbgm = np.sqrt(mean_squared_error(y_test, lgbm_y_predict))
print("RMSE: %f" % (rmse_lbgm))

# The Mean Squared Error
print("Mean squared error: %.2f" % mean_squared_error(y_test, lgbm_y_predict))

# R2 Score
print('Variance score: %.2f' % r2_score(y_test, lgbm_y_predict))

RMSE: 3.642627
Mean squared error: 13.27
Variance score: 0.85


### Observations:

● The RMSE score for the different models on the Test Data was as follows:<br>
    ○ Linear Regression - RMSE score = 4.671866<br>
    ○ Decision Trees - RMSE score = 5.423396 (was worse than Linear Model)<br>
    ○ Random Forest Regressor - RMSE score = 3.733577 (drastic improvement from here on)<br>
    ○ XGB Regressor with Parameters - RMSE score after tuning the parameters few times = 3.611114<br>
    ○ LGBM Regressor - RMSE score = 3.810423 (worse than XBGR)<br>

● We can clearly see that XGBoost regressor turned out to be the best machine learning model which is capable of determining the taxi fare with most accuracy.

● Real world datasets are extremely complicated and visualizing this kind of data in order to understand the data better requires efforts. In other words, a simple linear model is not always sufficient to best model a data BUT should definitely be the way to start as it gives us a better understanding into the data spread and feature dependencies.

In [27]:
result_df = pd.DataFrame(data={'Model': ['Linear regression', 'Decision Tree', 'Random forest', 'XGBoost', 'Light GBM'], 'RMSE': [rmse, rmse_dt, rmse_rf, rmse_xgbr2, rmse_lbgm], 'MSE': [mean_squared_error(y_test, lr_y_pred), mean_squared_error(y_test, dt_y_pred), mean_squared_error(y_test, rf_y_pred), mean_squared_error(y_test, xgbr2_y_pred), mean_squared_error(y_test, lgbm_y_predict)], 'R2 score': [r2_score(y_test, lr_y_pred), r2_score(y_test, dt_y_pred), r2_score(y_test, rf_y_pred), r2_score(y_test, xgbr2_y_pred), r2_score(y_test, lgbm_y_predict)]})
result_df

,Model,RMSE,MSE,R2 score
0,Linear regression,4.515310,20.388026,0.770483
1,Decision Tree,5.224722,27.297721,0.692697
2,Random forest,3.573594,12.770577,0.856236
3,XGBoost,3.464686,12.004050,0.864865
4,Light GBM,3.642627,13.268730,0.850628


In [28]:
# pickle.dump(grid_search, open("xgb_model.pkl", "wb"))